In [1]:
rdd = sc.parallelize(["This","is","how","I","create","an","rdd"])

In [61]:
rdd.count()

7

In [18]:
from pyspark.sql import Row
df = spark.createDataFrame(rdd.map(lambda w: Row(w)),["word"])

In [19]:
df

DataFrame[word: string]

In [31]:
lines = sc.textFile("/Users/alexlanda/work/raw_data/tolstoy.txt") 

In [34]:
sorted(lines.flatMap(lambda l: [(w,1) for w in l.split()]
             ).reduceByKey(lambda c1,c2: c1 + c2).collect(), key= lambda kv: -1*kv[1])

[(u'the', 31613),
 (u'and', 20535),
 (u'to', 16306),
 (u'of', 14797),
 (u'a', 10007),
 (u'in', 8197),
 (u'his', 7630),
 (u'he', 7626),
 (u'that', 7225),
 (u'was', 7187),
 (u'with', 5482),
 (u'had', 5304),
 (u'not', 4403),
 (u'at', 4192),
 (u'her', 3882),
 (u'as', 3694),
 (u'on', 3369),
 (u'it', 3328),
 (u'for', 3266),
 (u'I', 3207),
 (u'is', 2985),
 (u'but', 2787),
 (u'him', 2722),
 (u'The', 2543),
 (u'she', 2539),
 (u'from', 2512),
 (u'you', 2411),
 (u'said', 2407),
 (u'were', 2353),
 (u'by', 2314),
 (u'be', 2301),
 (u'all', 2264),
 (u'which', 1919),
 (u'have', 1906),
 (u'who', 1904),
 (u'one', 1712),
 (u'they', 1707),
 (u'He', 1667),
 (u'what', 1624),
 (u'this', 1577),
 (u'an', 1558),
 (u'or', 1512),
 (u'Prince', 1498),
 (u'so', 1464),
 (u'been', 1460),
 (u'did', 1414),
 (u'their', 1407),
 (u'would', 1335),
 (u'up', 1295),
 (u'Pierre', 1260),
 (u'are', 1157),
 (u'only', 1139),
 (u'could', 1089),
 (u'my', 1081),
 (u'when', 1056),
 (u'out', 1043),
 (u'into', 997),
 (u'no', 997),
 (u'if

In [39]:
sorted(lines.flatMap(lambda l: [(w,1) for w in l.split()]
             ).reduceByKeyLocally(lambda c1,c2: c1 + c2).iteritems(), key = lambda kv: -1*kv[1])

[(u'the', 31613),
 (u'and', 20535),
 (u'to', 16306),
 (u'of', 14797),
 (u'a', 10007),
 (u'in', 8197),
 (u'his', 7630),
 (u'he', 7626),
 (u'that', 7225),
 (u'was', 7187),
 (u'with', 5482),
 (u'had', 5304),
 (u'not', 4403),
 (u'at', 4192),
 (u'her', 3882),
 (u'as', 3694),
 (u'on', 3369),
 (u'it', 3328),
 (u'for', 3266),
 (u'I', 3207),
 (u'is', 2985),
 (u'but', 2787),
 (u'him', 2722),
 (u'The', 2543),
 (u'she', 2539),
 (u'from', 2512),
 (u'you', 2411),
 (u'said', 2407),
 (u'were', 2353),
 (u'by', 2314),
 (u'be', 2301),
 (u'all', 2264),
 (u'which', 1919),
 (u'have', 1906),
 (u'who', 1904),
 (u'one', 1712),
 (u'they', 1707),
 (u'He', 1667),
 (u'what', 1624),
 (u'this', 1577),
 (u'an', 1558),
 (u'or', 1512),
 (u'Prince', 1498),
 (u'so', 1464),
 (u'been', 1460),
 (u'did', 1414),
 (u'their', 1407),
 (u'would', 1335),
 (u'up', 1295),
 (u'Pierre', 1260),
 (u'are', 1157),
 (u'only', 1139),
 (u'could', 1089),
 (u'my', 1081),
 (u'when', 1056),
 (u'out', 1043),
 (u'into', 997),
 (u'no', 997),
 (u'if

In [40]:
from pyspark.ml.feature import StopWordsRemover, Tokenizer



In [45]:
tolstoy_df = spark.read.text("/Users/alexlanda/work/raw_data/tolstoy.txt").withColumnRenamed("value","lines")
tolstoy_df

DataFrame[lines: string]

In [60]:
from pyspark.sql.functions import *
tokenizer = Tokenizer(inputCol="lines", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="cleaned")
(remover.transform(tokenizer.transform(tolstoy_df))
    .select(explode(col("cleaned")).alias("word"))
     .filter(length(col("word"))> 3)
     .groupBy("word")
     .count()
     .orderBy("count",ascending=False)
     .show(truncate=False))

+--------+-----+
|word    |count|
+--------+-----+
|said    |2407 |
|prince  |1704 |
|would   |1341 |
|pierre  |1260 |
|could   |1094 |
|went    |846  |
|him.    |826  |
|princess|726  |
|him,    |716  |
|andrew  |700  |
|like    |694  |
|natasha |683  |
|french  |671  |
|came    |662  |
|without |660  |
|began   |653  |
|time    |644  |
|looked  |642  |
|know    |626  |
|still   |611  |
+--------+-----+
only showing top 20 rows



In [54]:
flights_df = spark.read.csv("/Users/alexlanda/work/raw_data/flights.csv", header=True, inferSchema=True).cache()

In [55]:
flights_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Ca

In [56]:
flights_df.registerTempTable("flights")
spark.sql("select Origin,Dest,count(*) as nflights from flights group by Origin,Dest order by nflights desc ").show()

+------+----+--------+
|Origin|Dest|nflights|
+------+----+--------+
|   LAX| OAK|      77|
|   HOU| DAL|      75|
|   DAL| HOU|      72|
|   LAS| PHX|      69|
|   OAK| LAX|      58|
|   LAX| LAS|      54|
|   LAS| OAK|      54|
|   LAS| SAN|      54|
|   LAS| LAX|      53|
|   SAN| OAK|      51|
|   LAX| PHX|      50|
|   LAS| BUR|      50|
|   PHX| LAS|      50|
|   OAK| SAN|      50|
|   OAK| BUR|      48|
|   LAS| RNO|      47|
|   BUR| OAK|      47|
|   LAS| MDW|      46|
|   PHX| SAN|      40|
|   OAK| LAS|      40|
+------+----+--------+
only showing top 20 rows

